In [35]:
from mtcnn import MTCNN
import cv2
import os
import pandas as pd
import tensorflow as tf

In [38]:
data = pd.read_json("./data/train_sample_videos/metadata.json").T
data

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
eudeqjhdfd.mp4,REAL,train,None


In [47]:
def find_faces_in_videos(video_directory, save_directory, video_name):
    cap = cv2.VideoCapture(f'{video_directory}{video_name}')
    detector = MTCNN()

    def get_frame(sec, current_count, name):
        cap.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
        hasFrames,image = cap.read()
        if hasFrames:
            result = detector.detect_faces(image)
            
            # Check if confidence is greater than 0.99
            for face in result:
                if face['confidence'] > 0.99:
                    current_count += 1
                    x1, y1, width, height = face['box']
                    x2, y2 = x1 + width, y1 + height
                    face_image = image[y1:y2, x1:x2]

                    # Resize image
                    face_image = cv2.resize(face_image, (128, 128))

                    # Ensure directory exists
                    label = data.loc[video_name, 'label']
                    if not os.path.exists(f'{save_directory}{label}'):
                        os.makedirs(f'{save_directory}{label}')

                    # Find label of person
                    # Save image 
                    cv2.imwrite(f"{save_directory}{label}/{name}{count}.jpg", face_image)
        return hasFrames, current_count

    sec = 0
    frame_rate = 2 #//it will capture image in each second
    count = 0
    name = video_name.replace(".mp4", "")
    success, count = get_frame(sec, 0, name)
    while success:
        success, count = get_frame(sec, count, name)
        sec += frame_rate

In [48]:
# Read frames from video
path = "./data/train_sample_videos/"
# Get all files in directory
files = os.listdir(path)
# Only keep .mp4 files
files = [f for f in files if f.endswith(".mp4")]
for file in files:
    find_faces_in_videos(path, "./data/processed/", file)


In [ ]:
batch_size = 32
img_height = 128
img_width = 128
data_dir = "./data/processed/"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)